# Importing useful modules

In [ ]:
%matplotlib inline

In [ ]:
from __future__ import division #Importer ça avant toute chose, autrement ne marche pas
import graphlab as gl
#import pandas as pd
gl.canvas.set_target('ipynb') # Pour afficher les graphes dans mon notebook plutôt que new tab

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from pylab import pcolor, show, colorbar, xticks, yticks
import seaborn as sns

In [ ]:
def sum(a,b):
    print a+b
    
my_tuple = (1,2)
sum(*my_tuple)

# Functions definitions

In [ ]:
#plot_corr(df_data_viz)
corr = df.corr()
pcolor(corr)
colorbar()
yticks(np.arange(0.5,6),range(0,10))
xticks(np.arange(0.5,6),range(0,10))
show()

In [ ]:
def plot_correlation_map( df ):
    corr = df.corr()
    _ , ax = plt.subplots( figsize =( 12 , 10 ) )
    cmap = sns.diverging_palette( 220 , 10 , as_cmap = True )
    _ = sns.heatmap(
        corr, 
        cmap = cmap,
        square=True, 
        cbar_kws={ 'shrink' : .9 }, 
        ax=ax, 
        annot = True, 
        annot_kws = { 'fontsize' : 12 }
    )

In [ ]:
plot_correlation_map(df_data_viz)

In [ ]:
corr = df.corr()
corr #shape = (6,6)

In [ ]:
df_data_validation.loc[(df_data_validation.Pclass == 1), "Pclass1"] = 1
df_data_validation.loc[(df_data_validation.Pclass == 2), "Pclass2"] = 1
df_data_validation.loc[(df_data_validation.Pclass == 3), "Pclass3"] = 1
df_data_validation = df_data_validation.fillna(0)

## Check data imbalance

Can we improve the result with a better balancing of the data ?

### Going back to Graphlab SFrames

In [ ]:
data = gl.SFrame(df_data)
data

In [ ]:
data_for_validation = gl.SFrame(df_data_validation)
data_for_validation

# Defining features, target 

In [ ]:
#features = ["Sex", "Pclass1","Pclass2","Pclass3", "Is_baby", "Age"]
features = ["Pclass", "Fare"]
target = "Survived"
features

## Splitting data into training and validation sets

# Model

In [ ]:
model = tree_model(train_data,features,MLR=0.2, MCW=0)

In [ ]:
#Optimizing min_loss_reduction
np.linspace(0,1,num=10)

In [ ]:
train_acc = []
valid_acc = []
end = 2
nb = 100
mlr=1
for mcw in np.linspace(0,end,nb):
    model_MLR = tree_model(train_data,features,mlr,mcw)
    #print "Training accuracy", model_MLR.get('training_accuracy')
    prediction_validation = model.predict(validation_data)
    #print "Validation accuracy", calc_accuracy(prediction_validation,validation_data)
    train_acc.append(model_MLR.get('training_accuracy'))
    valid_acc.append(calc_accuracy(prediction_validation,validation_data))

In [ ]:
plt.plot(np.linspace(0,end,nb),train_acc,'*')

In [ ]:
plt.plot(np.linspace(0,end,nb),valid_acc,'*')

In [ ]:
prediction_validation = model.predict(validation_data)
prediction_validation

In [ ]:
model.show(view = "Tree")

In [ ]:
model_MLR.show(view="Tree")

## Estimating error on validation data

#### Error / accuracy

In [ ]:
calc_accuracy(prediction_validation,validation_data)

# Prediction using the validation data without ground truth

In [ ]:
prediction = model.predict(data_for_validation)

# Working with cross validation

First, shuffle data

In [ ]:
data_shuffled = gl.cross_validation.shuffle(data)

We split the data into a k-set of train/validation data (here k = 5).

folds is a kx2 array where folds[k][0] is the train set and folds[k][1] the validation set

In [ ]:
k = 10
folds = gl.cross_validation.KFold(data_shuffled,k)

We can check that the length of one fold is that of the total data

In [ ]:
print "Size of train/validation set in the fold", len(folds[0][0])+len(folds[0][1])
print "Size of the initial data", len(data)


In [ ]:
acc_sum=0
for train,valid in folds:
    #m = gl.decision_tree_classifier.create(train,target="Survived",validation_set=None,verbose=None)
    m = gl.decision_tree_classifier.create(train,target="Survived",validation_set=valid,verbose=None
                                          , min_loss_reduction = 0.2,min_child_weight = 0 )
    #print "EVALUATION", m.evaluate(valid) #Direct evaluation of the model on the validation set
    #print "PREDICTION", m.predict(valid) #Prediction of survival using the model
    #accuracy = m.evaluate(valid, metric='accuracy')
    #acc = accuracy['accuracy'] #accuracy is a dict so we need this step to extract a float
    #acc = calc_accuracy(m.predict(valid),valid)
    acc = m.get("validation_accuracy")
    acc_sum+=acc
    print("Accuracy of the model : %.6f" %acc)

print "Average accuracy on %0.f validation sets: %f " %(k ,(acc_sum/k))

# Exporting result as a csv file for submission

Plot of the correlation matrix

In [ ]:
df_data_viz = pd.DataFrame(df_data)
del df_data_viz["Sex"]
del df_data_viz["Is_baby"]
#del df_data_viz["Pclass1"]
#del df_data_viz["Pclass2"]
#del df_data_viz["Pclass3"]

In [ ]:
df_data_viz.head()

In [ ]:
plot_categories(df_data, cat='Sex', target="Survived")

# Performing some data manipulation

In [ ]:
#Let's remove unused columns for clarity (not necessary)
del df_data["Name"]
#del df_data["SibSp"]
#del df_data['Parch']
del df_data["Ticket"]
del df_data["Cabin"]
del df_data["Embarked"]
del df_data_validation["Name"]
#del df_data_validation["SibSp"]
#del df_data_validation['Parch']
del df_data_validation["Ticket"]
del df_data_validation["Cabin"]
del df_data_validation["Embarked"]

### Let's examine the feature "Age" for which we have to fill in the missing values

Age is not defined for all data points, we have to find a way to deal with this feature.
### Let's try a simple aproach
* if fare is 0 and age is not defined then probably we have a baby and age=0
* if age is <1 then age = 0 (baby)

Do we still have Nan values ?

In [ ]:
df_data["Age"] = df_data["Age"].apply(age_change)
df_data_validation["Age"] = df_data_validation["Age"].apply(age_change)

Some passenger did not pay for the travel (Fare==0) and age is not defined, they probably are babies so let's change their age to 0.

In [ ]:
df_data_validation[df_data_validation["Age"].isnull() & (df_data_validation["Fare"]==0)]

In [ ]:
df_data[df_data["Age"].isnull() & (df_data["Fare"]==0)]

In [ ]:
df_data.loc[(df_data.Age.isnull()) & (df_data["Fare"] == 0), "Age"] = 0

In [ ]:
df_data_validation.loc[(df_data_validation.Age.isnull()) & (df_data_validation["Fare"] == 0), "Age"] = 0

Then we will fill the remaining missing values with the mean of the age. BUT, I make the assumption that the mean age will depend on the class (rich people in 1st class will probably be older)

In [ ]:
mean_age_Class1 = round(df_data["Age"][df_data["Pclass"]==1].mean()) #38 
mean_age_Class2 = round(df_data["Age"][df_data["Pclass"]==2].mean()) #30
mean_age_Class3 = round(df_data["Age"][df_data["Pclass"]==3].mean()) #25

In [ ]:
df_data.loc[(df_data.Age.isnull()) & (df_data["Pclass"] == 1), "Age"] = mean_age_Class1
df_data.loc[(df_data.Age.isnull()) & (df_data["Pclass"] == 2), "Age"] = mean_age_Class2
df_data.loc[(df_data.Age.isnull()) & (df_data["Pclass"] == 3), "Age"] = mean_age_Class3

df_data["Is_baby"] = 0

df_data.loc[(df_data.Age < 4), "Is_baby"] = 1

What are the mean ages for the validation set ?

In [ ]:
mean_age_Class1_valid = round(df_data_validation["Age"][df_data_validation["Pclass"]==1].mean()) #41 
mean_age_Class2_valid = round(df_data_validation["Age"][df_data_validation["Pclass"]==2].mean()) #29
mean_age_Class3_valid = round(df_data_validation["Age"][df_data_validation["Pclass"]==3].mean()) #24

In [ ]:
df_data_validation.loc[(df_data_validation.Age.isnull()) & (df_data_validation["Pclass"] == 1), "Age"] = mean_age_Class1_valid
df_data_validation.loc[(df_data_validation.Age.isnull()) & (df_data_validation["Pclass"] == 2), "Age"] = mean_age_Class2_valid
df_data_validation.loc[(df_data_validation.Age.isnull()) & (df_data_validation["Pclass"] == 3), "Age"] = mean_age_Class3_valid

df_data_validation["Is_baby"] = 0

df_data_validation.loc[(df_data.Age < 4), "Is_baby"] = 1

Let's create some categorical features

In [ ]:
df_data.loc[(df_data.Pclass == 1), "Pclass1"] = 1
df_data.loc[(df_data.Pclass == 2), "Pclass2"] = 1
df_data.loc[(df_data.Pclass == 3), "Pclass3"] = 1
df_data = df_data.fillna(0)

In [ ]:
df_data_validation = pd.read_csv("./test.csv") #Data used for submission to Kaggle
data_validation_raw = gl.SFrame.read_csv("./test.csv")

# Let's look at the data

In [ ]:
data_raw.show()

In [ ]:
data_raw["Pclass"].show(view = "Categorical")

In [ ]:
data_raw["Sex"].show()

In [ ]:
Similar operations are done on the validation set

In [ ]:
plot_categories( df_data , cat = 'Pclass' , target = 'Survived' )

In [ ]:
def plot_categories( df , cat , target , **kwargs ):
    row = kwargs.get( 'row' , None )
    col = kwargs.get( 'col' , None )
    facet = sns.FacetGrid( df , row = row , col = col )
    facet.map( sns.barplot , cat , target )
    facet.add_legend()

In [ ]:
def plot_corr(df,size=8):
    '''Function plots a graphical correlation matrix for each pair of columns in the dataframe.

    Input:
        df: pandas DataFrame
        size: vertical and horizontal size of the plot'''

    corr = df.corr()
    fig, ax = plt.subplots(figsize=(size, size))
    ax.matshow(corr)
    pcolor(corr)
    colorbar()
    plt.xticks(range(len(corr.columns)), corr.columns);
    plt.yticks(range(len(corr.columns)), corr.columns);

In [ ]:
#prediction_validation is the prediction done on the validation data using the validation set
#validation_data is the validation set
def calc_accuracy(prediction_validation, validation_data):
    accuracy = ((prediction_validation==validation_data["Survived"]).sum())/len(validation_data)
    return accuracy
#print("Accuracy of the model : %.3f" %accuracy)

In [ ]:
#All babies should have age 0
def age_change(age):
    if age<=1:
        return 0
    else:
        return age

In [ ]:
def tree_model(data, features,MLR,MCW):
    return gl.decision_tree_classifier.create(data,features = features, target = target, 
                                              validation_set=None, min_loss_reduction = MLR,
                                             verbose = None, min_child_weight = MCW, max_depth=3)

# Importing csv files

In [ ]:
#Let's work with pandas then import in graphlab
df_data = pd.read_csv("./train.csv") #My training data
data_raw = gl.SFrame.read_csv("./train.csv")